<a href="https://colab.research.google.com/github/AlbertNg02/AlbertNg02.github.io/blob/master/Lab2_SolvingMDPswithDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import numpy as np
import matplotlib.pyplot as plt

import unittest
import functools

from collections import namedtuple

In [69]:
# install and load testing libraries
####################################

# suppresses output generated during package installation
%%capture

%pip install ipython_unittest;
%pip install jupyter_dojo;

%load_ext ipython_unittest

# Solving MDPs with Dynamic Programming

In this lab, we will formalize two sequential decision problems as Markov decision processes (MDPs) and solve them using techniques from dynamic programming that are based on the Bellman equations we discussed in class.

The lab will walk you through a set of stages, during which you will implement the iterative algorithms described in Chapter 4 of (Sutton & Barto, 2020). These algorithms include,

1. Policy Evaluation,
2. Policy Improvement, and
3. Value Iteration.

## Markov Decision Process #1 - 5x5 Grid World

We'll start by defining a Markov decision process (MDP) for a simple, deterministic, grid world problem.



<img src="https://drive.google.com/uc?export=view&id=1X9EmWHzYqfI9TAN8R5AfE5uJIec0EnTG" width=900 height=400/>

Recall from chapter 3 of Sutton & Barto that an MDP is defined by a tuple of elements, $\langle\mathcal{S}, \mathcal{A}, p, r, \gamma\rangle$.

* $\mathcal{S}$ is the set of all states.
* $\mathcal{A}$ is the set of all actions.
* $p$ is function that defines the environment's dynamics.
* $r$ is a reward function that gives the immediate rewards for each state transition, and
*$\gamma$ is a discount factor ($0.0 \le \gamma \le 1.0$) that quantifies the value of future rewards.

We need to define each of these MDP elements for our problem.

In [70]:
# our goal is to populate this data structure
MDP = namedtuple('MDP', ['S', 'A', 'p', 'r', 'gamma'])

### States

Let's start by defining the states.

In [71]:
# grid size
n_rows, n_cols = 5, 5

# complete list of states
states = np.arange(n_rows * n_cols)

# convenience aliases used when defining p() and r()
s_goal = [9]       # treasure
s_death = [12, 19] # pits
s_term = np.array([*s_goal, *s_death])

### Actions

Now, we'll define the actions.

In [72]:
# complete list of actions
actions = N,S,E,W = np.arange(4)

# used later for pretty printing
action_map=lambda x: ['N', 'S', 'E', 'W'][x]

### Environment Dynamics: $p(s' | s, a)$

Next, we'll specify the environment's dynamics, $p(s'|s,a)$.

In general, $p(\cdot)$ would also be parameterized on the immediate rewards, $r$. However, since this environment's state transitions AND rewards are *deterministic*, we can simplify $p(\cdot)$ implementation by leaving off $r$ as a parameter. That is, since there is only a single possible immediate reward, $r$, for each combination of $s$, $s'$, and $a$, so don't need it.

To make our lives even easier, we will define a helper function `hit_wall()` that we can use to calculate some of the probabilties returned from our implementation of `p()`. Specifically, whenever we hit a wall, the current and subsequent states should be the same with probability 1.0.

In [73]:
# helper function

def hit_wall(s, a):
  """ Returns true if taking an action a in state s would hit a wall.

  Args:
    s: the state that action a is taken from
    a: the action taken

  Returns:
    True if agent would hit a wall and False otherwise.
  """
  # north:
  if a == N and s in [0, 1, 2, 3, 4, 6, 7, 8, 13, 17, 21, 22, 23, 24]:
    return True

  # south:
  if a == S and s in [1, 2, 3, 7, 8, 16, 17, 18, 20, 21, 22, 23, 24]:
    return True

  # east:
  if a == E and s in [4, 8, 10, 11, 14, 15, 24]:
    return True

  # west:
  if a == W and s in [0, 5, 10, 11, 15, 16, 20]:
    return True

  return False

Now, let's implement `p()`.

In [74]:
# environment dynamics - pay careful attention to the order of the parameters
def p(s, s_prime, a):
  """ Returns the probability of moving from one state to another for an action.

  The dynamics are deterministic for this environment, so the returned
  values are either 1.0 (legal move) or 0.0 (illegal move).

  Args:
    s_prime: the successor state resulting from the action
    s: the state in which the action was taken
    a: the action taken

  Returns:
    the transition probability, p(s'|s,a) in {0.0, 1.0}
  """

  # same state transitions only possible for terminal states and hitting walls
  if hit_wall(s, a) or s in s_term:
    return 1.0 if s == s_prime else 0.0

  # not (hitting walls or terminal state)
  else:
    if (a == N) and s_prime == s - n_cols:
      return 1.0

    if (a == S) and s_prime == s + n_cols:
      return 1.0

    if (a == E) and s_prime == s + 1:
      return 1.0

    if (a == W) and s_prime == s - 1:
      return 1.0

  # illegal transition
  return 0.0

### Reward Function: $r(s, a, s')$

Finally, we define our reward function, $r(\cdot)$.

For this problem, we have the following immediate rewards,
1. transitioning into a **goal state** (that is, getting treasure) yields a reward of 25.0.
2. transitioning into a **pit state** (that is, death) yields a reward of -100.0, and
3. transitioning into **any other state** yields a reward of -1.0.

That is,

* $r_{goal}=25.0$
* $r_{death}=-100.0$
* $r_{other}=-1.0$


In [75]:
def r(s, s_prime, a):
  """ Returns the reward for a single environmental interaction.

  Args:
    s: the state before the action was taken
    s_prime: the state after the action was taken
    a: the action taken

  Returns:
    an immediate reward based on this environmental interaction
  """
  # reward when transitioning into the goal state
  r_goal = 25.0

  # reward when transitioning into a pit
  r_death = -100.0

  # reward for all other transitions
  r_other = -1.0

  # if current state is terminal then all actions generate zero rewards
  if s in s_term:
    return 0.0

  # transition to goal states
  if s_prime in s_goal and s != s_prime:
    return r_goal

  # transition to death states
  if s_prime in s_death and s != s_prime:
    return r_death

  return r_other

Finally, let's gather all of those pieces and define our MDP.

In [76]:
mdp = MDP(states, actions, p, r, 0.9)

Now let's go about the business of solving MDPs using dynamic programming!

## Policy Iteration

The Policy Iteration algorithm (see Sutton & Barto, Section 4.2) is a dynamic programming algorithm that applies an iterative update based on the Bellman Expectation Equation (see Sutton & Barto, Figure 3.14) to discover an optimal policy for an MDP.

Specifically, the Policy Iteration algorithm involves the repeated application of two operations:

1. **Policy Evaluation** to approximate the state-value function $v_k \approx v_\pi$.
2. **Policy Improvement** to define a new policy, $\pi_k$, by acting greedily with respect $v_k$.

These steps are repeated until both the state-value estimates and the policy converge to the optimal policy ($\pi_0 → \pi_1 → \cdots \rightarrow \pi_*$) and its corresponding state-value function ($v_0 \rightarrow v_1 \rightarrow \cdots \rightarrow v_*$).

Note that for a finite MDP (that is, finite number of states and actions) this convergence is guaranteed.

We will be implementing each of the steps separately in multiple stages.

In [77]:
# helper functions for pretty printing outputs
##############################################

def display_state_values(V, decimals=2):
  """ Pretty prints the state-value for each state.

  Args:
    V: an array of state-value estimates

  Returns:
    None
  """
  print(np.round(V.reshape((n_rows, n_cols)), decimals))

def display_action_values(Q, decimals=2):
  """ Pretty prints the action-values for each state.

  Args:
    Q: an array of action-value estimates

  Returns:
    None
  """
  print(np.round(Q, decimals))


def display_policy(policy, action_map, term_string='-'):
  """ Pretty prints the best action per state for a policy (as a string).

  Note that terminals states are displayed with the character '-'.

  Args:
    policy: the policy whose actions will be displayed.
    action_map: a function that maps actions to strings
    term_string: a string to use for terminal states

  Returns:
    None
  """
  # selects the best action per state
  actions = np.argmax(Q, axis=1)

  # map actions onto action strings
  s_array = np.array(list(map(action_map, actions)))

  # replace actions for terminal states with term string
  s_array[s_term] = term_string

  # print out values in a n_rows x n_cols grid
  print(s_array.reshape((n_rows, n_cols)))

### Policy Evaluation

**You should review Section 4.1 in Sutton & Barto before continuing.**

Policy Evaluation is a dynamic programming algorithm that estimates the state-values (or action-values) for a policy using the Bellman Expectation Equation.

We call policy evaluation a **prediction** algorithm because it is only used to determine the value of a policy without trying to improve that policy.
<br><br>

#### Uniform Random Policy

We'll start by implementing a function for a uniform random policy. That is, a function that gives equal probability to select each action, regardless of the state.

In [78]:
def generate_uniform_random_policy(mdp):
  def uniform_random_policy(a, s):
    """ Returns the probability of taking action a from state s.

    Based on a uniform random policy.

    Args:
      s: state (0 <= s < len(states))
      a: action taken in state (0 <= a < len(actions))

    Returns:
      the probability of taking action a in state s (0.0 <= p <= 1.0)
    """
    return 1.0/len(mdp.A)
  return uniform_random_policy

Next, we'll implement a function that updates our array of estimates of the state-value function ($V_k$) based on the previous estimates ($V_{k-1}$).


#### State-Value Estimates Update Rule

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Implement the update rule used to iteratively improve our estimates of the state-value function!**

Hint: Implement the iterative update given in Section 4.1, Equation 4.5.

In [79]:
mdp

MDP(S=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24]), A=array([0, 1, 2, 3]), p=<function p at 0x79ebd42d39a0>, r=<function r at 0x79ebd42d2290>, gamma=0.9)

In [80]:
action_map(1)

'S'

In [81]:
def update_state_value_estimate(mdp, V_prev, policy):
  """ Returns an updated estimate of the state-values.

  Args:
    mdp: the MDP
    V_prev: an array of state-value estimates from the previous iteration
    policy: the policy being evaluated

  Returns:
    An array containing updated state-values for every state in the MDP.
  """
  S, A, p, r, gamma = mdp
  V = np.zeros(len(states))

  #################################
  # Add your implementation here! #
  #################################


  for s in S:
    val = 0
    for a in A:
        for s_prime in S:
            val += policy(a,s) * p(s, s_prime,a) * (r(s, s_prime, a) + gamma*V_prev[s_prime])
    V[s] = val


  return V

#### Solution

Bringing it all together!

In [82]:
def policy_evaluation(mdp, policy, update, V_init=None, theta=1e-5,
                      k_max=np.inf, verbose=True, debug=False):
  """ Approximates the state-value function for a policy.

  This is an iterative algorithm that terminates when the max absolute difference
  in the state-value estimates between iterations is less than theta OR when the
  max number of iterations (k_max) is reached.

  Args:
    mdp: the MDP
    policy: the policy being evaluated
    update: a function that updates V based on previous estimate
    V_init: initial approximation for the state-values
    theta: the max abs difference termination threshold
    k_max: the max number of iterations
    verbose: if True displays summary info per iteration
    debug: if True displays the current V estimates per iteration
  Returns:
    A tuple where the first element is the number of iterations (k) before
    termination/convergence and the second element is an array containing
    approximate state-values for every state in mdp, and the second element.
  """
  delta = np.inf # change between iterations
  k = 0

  V_prev = np.zeros(len(mdp.S)) if V_init is None else V_init

  while delta > theta and k < k_max:
    if verbose:
      print(f'beginning iteration {k}; delta: {delta}')

    # iteratively updates the state-value estimates
    V = update(mdp, V_prev, policy)

    if debug:
      display_state_values(V)

    # determine the max absolute of state-values between this iteration and
    # the last iteration
    delta = np.amax(np.abs(V - V_prev))

    # copy values to separate array (updates not in-place)
    V_prev[:] = V

    k += 1

  return k, V

Now, let's look at the first 5 iterations of our Policy Evaluation implementation on the uniform random policy with debugging output enabled.

This displays our state-value function estimates after each iteration.

Do the results make sense?
* Is `delta` decreasing on each iteration?
* Are the values of all states updating? (Except for the terminal states!)

In [83]:
uniform_random_policy = generate_uniform_random_policy(mdp)

# runs the policy evaluation algorithm, displaying the state-value
# approximations and delta values after each iteration
_ = policy_evaluation(mdp, uniform_random_policy,
                      update_state_value_estimate,
                      k_max=5, debug=True)

beginning iteration 0; delta: inf
[[ -1.    -1.    -1.    -1.     5.5 ]
 [ -1.    -1.    -1.    -1.     0.  ]
 [ -1.    -1.     0.   -25.75 -19.25]
 [ -1.    -1.    -1.   -25.75   0.  ]
 [ -1.    -1.    -1.    -1.    -1.  ]]
beginning iteration 1; delta: 25.75
[[ -1.9   -1.9   -1.9   -0.44   7.75]
 [ -1.9   -1.9   -1.9   -1.9    0.  ]
 [ -1.9   -1.9    0.   -41.67 -29.38]
 [ -1.9   -1.9   -7.47 -37.56   0.  ]
 [ -1.9   -1.9   -1.9   -1.9   -1.9 ]]
beginning iteration 2; delta: 15.918750000000003
[[ -2.71  -2.71  -2.38   0.12   8.89]
 [ -2.71  -2.71  -2.71  -2.71   0.  ]
 [ -2.71  -2.71   0.   -50.19 -35.23]
 [ -2.71  -3.96 -13.24 -45.26   0.  ]
 [ -2.71  -2.71  -2.71  -2.71  -2.71]]
beginning iteration 3; delta: 8.517656250000002
[[ -3.44  -3.36  -2.65   0.52   9.53]
 [ -3.44  -3.44  -3.44  -3.44   0.  ]
 [ -3.44  -3.72   0.   -55.15 -38.47]
 [ -3.44  -6.37 -18.03 -50.2    0.  ]
 [ -3.44  -3.44  -3.44  -3.44  -3.44]]
beginning iteration 4; delta: 4.966312500000001
[[ -4.08  -3.89  -2.8

What about if we change our MDP so that $\gamma=0.0$?

In [84]:
mdp_gamma_zero = mdp._replace(gamma=0.0)
_ = policy_evaluation(mdp_gamma_zero, uniform_random_policy,
                      update_state_value_estimate, k_max=5, debug=True)

beginning iteration 0; delta: inf
[[ -1.    -1.    -1.    -1.     5.5 ]
 [ -1.    -1.    -1.    -1.     0.  ]
 [ -1.    -1.     0.   -25.75 -19.25]
 [ -1.    -1.    -1.   -25.75   0.  ]
 [ -1.    -1.    -1.    -1.    -1.  ]]
beginning iteration 1; delta: 25.75
[[ -1.    -1.    -1.    -1.     5.5 ]
 [ -1.    -1.    -1.    -1.     0.  ]
 [ -1.    -1.     0.   -25.75 -19.25]
 [ -1.    -1.    -1.   -25.75   0.  ]
 [ -1.    -1.    -1.    -1.    -1.  ]]


What about if we change our MDP so that $\gamma=1.0$

In [85]:
mdp_gamma_one = mdp._replace(gamma=1.0)
_ = policy_evaluation(mdp_gamma_one, uniform_random_policy,
                      update_state_value_estimate, k_max=5, debug=True)

beginning iteration 0; delta: inf
[[ -1.    -1.    -1.    -1.     5.5 ]
 [ -1.    -1.    -1.    -1.     0.  ]
 [ -1.    -1.     0.   -25.75 -19.25]
 [ -1.    -1.    -1.   -25.75   0.  ]
 [ -1.    -1.    -1.    -1.    -1.  ]]
beginning iteration 1; delta: 25.75
[[ -2.    -2.    -2.    -0.38   8.  ]
 [ -2.    -2.    -2.    -2.     0.  ]
 [ -2.    -2.     0.   -43.44 -30.5 ]
 [ -2.    -2.    -8.19 -38.88   0.  ]
 [ -2.    -2.    -2.    -2.    -2.  ]]
beginning iteration 2; delta: 17.6875
[[ -3.    -3.    -2.59   0.31   9.41]
 [ -3.    -3.    -3.    -3.     0.  ]
 [ -3.    -3.     0.   -53.95 -37.73]
 [ -3.    -4.55 -15.31 -48.38   0.  ]
 [ -3.    -3.    -3.    -3.    -3.  ]]
beginning iteration 3; delta: 10.515625
[[ -4.    -3.9   -2.97   0.86  10.28]
 [ -4.    -4.    -4.    -4.     0.  ]
 [ -4.    -4.39   0.   -60.77 -42.17]
 [ -4.    -7.85 -21.89 -55.16   0.  ]
 [ -4.    -4.    -4.    -4.    -4.  ]]
beginning iteration 4; delta: 6.8125
[[ -4.97  -4.69  -3.24   1.26  10.86]
 [ -5.    -5.

Once you feel confident that the algorithm is working properly, let's execute it *until convergence* on the original mdp ($\gamma = 0.9$).

In [86]:
k, V = policy_evaluation(mdp, uniform_random_policy, update_state_value_estimate, verbose=False)

Now let's do the same for a modified version of the MDP with $\gamma=1.0$.

**Warning: The following cell can take a while to run (approximately 40 seconds).**

In [87]:
k_gamma_one, V_gamma_one = policy_evaluation(mdp_gamma_one, uniform_random_policy, update_state_value_estimate, verbose=False)

In [88]:
print(f'(iterations when gamma = 0.9): {k}\n(iterations when gamma = 1.0): {k_gamma_one}')

(iterations when gamma = 0.9): 110
(iterations when gamma = 1.0): 2463


In [89]:
display_state_values(V, decimals=5)

[[ -9.24397  -7.53782  -4.73738   0.40199  10.16445]
 [-10.61413 -12.55234 -11.45668 -11.00845   0.     ]
 [-10.31905 -16.72061   0.      -64.00882 -43.42192]
 [-10.1658  -23.87584 -37.19812 -62.60847   0.     ]
 [-10.08627 -10.04511 -10.02404 -10.01368 -10.00945]]


In [140]:
display_state_values(V_gamma_one, decimals=3)

[[-174.132 -142.105 -106.079  -66.053  -22.026]
 [-202.158 -198.185 -206.185 -210.184    0.   ]
 [-230.158 -182.211    0.     -89.609  -55.536]
 [-254.157 -162.238 -138.265 -110.291    0.   ]
 [-274.157 -290.157 -302.157 -310.157 -314.157]]


What do you notice about the state-value functions (`V` and `V_gamma_one`) and the number of iterations until convergence (`k` and `k_gamma_one`) between these two executions of Policy Evaluation?

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**What explains these differences? Write your observations in the text cell below.**

`<YOUR ANSWER HERE>` TODO

#### Verifying Our Solution

Now, let's write a function to verify that the state-values are correct.



<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Implement a function to verify that the state-value approximations are correct.**

*Hint: Read Sutton & Barto, p. 59 carefully.*

In [154]:
def verify_state_values(mdp, V, policy, tolerance=1e-5):
  """ Verifies that V satisfies the Bellman expectation equation.

  Args:
    V: array estimates of the state-value function v_pi
    policy: the policy that was used to calculate V's values
    tolerance: absolute tolerance on equality check

  Return:
    True if all values in V satisfy the Bellman equation; False otherwise.

  """
  #################################
  # Add your implementation here! #
  #################################
  S, A, p, r, gamma = mdp

  for s in S:
     val = 0
     for a in A:
        for s_prime in S:
            if val == 0:
                val = policy(a,s) * p(s, s_prime,a) * (r(s, s_prime, a) + gamma* V[s_prime])
            if not (abs(val - V[s]) < tolerance):
                return False
  return True





In [157]:
# do not change this line!
%%unittest
assert verify_state_values(mdp, V, uniform_random_policy)

Fail

F
FAIL: test_1 (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 1, in test_1
AssertionError: False is not true

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

**Warning: The following cell can take a while to run (approximately 40 seconds).**

In [158]:
%%unittest_main
class TestPolicyEvaluation(unittest.TestCase):

    # precision threshold
    abs_threshold = 1e-5

    def test_gamma_zero(self):
      mdp_gamma_zero = mdp._replace(gamma=0.0)

      V_expected = np.array(
          [-1.0,    -1.0,    -1.0,    -1.0,     5.5,
           -1.0,    -1.0,    -1.0,    -1.0,     0.0,
           -1.0,    -1.0,     0.0,   -25.75, -19.25,
           -1.0,    -1.0,    -1.0,   -25.75,   0.0,
           -1.0,    -1.0,    -1.0,    -1.0,    -1.0]
          )

      _, V_actual = policy_evaluation(mdp_gamma_zero,
                                      uniform_random_policy,
                                      update_state_value_estimate,
                                      theta = TestPolicyEvaluation.abs_threshold,
                                      verbose=False)

      np.testing.assert_almost_equal(V_expected, V_actual, decimal=3)

    def test_gamma_one(self):
      mdp_gamma_one = mdp._replace(gamma=1.0)

      V_expected = np.array(
          [-174.132, -142.105, -106.079,  -66.053,  -22.026,
           -202.158, -198.185, -206.185, -210.184,      0.0,
           -230.158, -182.211,      0.0,  -89.609,  -55.536,
           -254.157, -162.238, -138.265, -110.291,      0.0,
           -274.157, -290.157, -302.157, -310.157, -314.157]
          )

      _, V_actual = policy_evaluation(mdp_gamma_one,
                                      uniform_random_policy,
                                      update_state_value_estimate,
                                      theta = TestPolicyEvaluation.abs_threshold,
                                      verbose=False)

      np.testing.assert_almost_equal(V_expected, V_actual, decimal=3)

    def test_gamma_orig(self):
      V_expected = np.array(
          [-9.244,  -7.538,  -4.737,   0.402,  10.164,
          -10.614, -12.552, -11.457, -11.008,     0.0,
          -10.319, -16.721,     0.0, -64.009, -43.422,
          -10.166, -23.876, -37.198, -62.608,     0.0,
          -10.086, -10.045, -10.024, -10.014, -10.009]
          )

      _, V_actual = policy_evaluation(mdp,
                                      uniform_random_policy,
                                      update_state_value_estimate,
                                      theta = TestPolicyEvaluation.abs_threshold,
                                      verbose=False)

      np.testing.assert_almost_equal(V_expected, V_actual, decimal=3)

Fail

FFF
FAIL: test_gamma_one (__main__.TestPolicyEvaluation)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 42, in test_gamma_one
  File "/usr/lib/python3.10/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/numpy/testing/_private/utils.py", line 521, in assert_almost_equal
    return assert_array_almost_equal(actual, desired, decimal, err_msg)
  File "/usr/lib/python3.10/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/numpy/testing/_private/utils.py", line 1034, in assert_array_almost_equal
    assert_array_compare(compare, x, y, err_msg=err_msg, verbose=verbose,
  File "/usr/lib/python3.10/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/numpy/testing/_private/utils.py", line 797, in assert_array_compare
    raise A

<unittest.runner.TextTestResult run=3 errors=0 failures=3>

### Policy Improvement

In this section you will implement the Policy Improvement step of the Policy Iteration algorithm.

We will do this in stages.

1. Calculate the action-value estimates, $Q \approx q_\pi(s,a)$, from the state-value estimates, $V \approx v_\pi(s)$.
2. Define a greedy policy based on $Q$.
3. Identify whether two policies are "stable" (the convergence criterion)

#### Estimating action-values ($Q$) from $V$

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Calculate the array of the action-value estimates ($Q$) from the array of state-value estimates ($V$)!**

Hint: See Sutton & Barto, p. 79, Equation 4.9.

In [220]:
def calc_Q(mdp, V):
  """ Calculates an array of action-values from state-value estimates.

  Args:
    mdp: the MDP
    V: an array of state-value estimates

  Return:
    An array of action-value estimates.
  """
  S, A, p, r, gamma = mdp
  Q = np.zeros(shape=(len(states), len(actions)))

  #################################
  # Add your implementation here! #
#################################

  for s in S:
    val = 0
    for a in A:
        for s_prime in S:
            Q[s][a] += p(s, s_prime, a) * (r(s,s_prime, a) + gamma * V[s_prime])



#   for s in S:
#     val = 0
#     for a in A:
#         for s_prime in S:
#             val += policy(a,s) * p(s, s_prime,a) * (r(s, s_prime, a) + gamma*V_prev[s_prime])
#     V[s] = val


  return Q

In [221]:
mdp

MDP(S=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24]), A=array([0, 1, 2, 3]), p=<function p at 0x79ebd42d39a0>, r=<function r at 0x79ebd42d2290>, gamma=0.9)

In [222]:
Q = calc_Q(mdp, V)
Q

array([[   0.,   -1.,   -1.,    0.],
       [   0.,   -1.,   -1.,   -1.],
       [   0.,   -1.,   -1.,   -1.],
       [   0.,   -1.,   -1.,   -1.],
       [   0.,   25.,   -1.,   -1.],
       [  -1.,   -1.,   -1.,   -1.],
       [  -1.,   -1.,   -1.,   -1.],
       [  -1., -100.,   -1.,   -1.],
       [  -1.,   -1.,   25.,   -1.],
       [   0.,    0.,    0.,    0.],
       [  -1.,   -1.,   -1.,   25.],
       [  -1.,   -1., -100.,   -1.],
       [   0.,    0.,    0.,    0.],
       [  -1.,   -1.,   -1., -100.],
       [  25., -100.,   -1.,   -1.],
       [  -1.,   -1.,   -1.,   -1.],
       [  -1.,   -1.,   -1.,   -1.],
       [-100.,   -1.,   -1.,   -1.],
       [  -1.,   -1., -100.,   -1.],
       [   0.,    0.,    0.,    0.],
       [  -1.,    0.,   -1., -100.],
       [  -1.,    0.,   -1.,   -1.],
       [  -1.,    0.,   -1.,   -1.],
       [  -1.,    0.,   -1.,   -1.],
       [-100.,    0.,    0.,   -1.]])

In [223]:
%%unittest_testcase
def test_calcQ(self):
  V_test = np.array(
      [-1.0,    -1.0,    -1.0,    -1.0,     5.5,
        -1.0,    -1.0,    -1.0,    -1.0,     0.0,
        -1.0,    -1.0,     0.0,   -25.75, -19.25,
        -1.0,    -1.0,    -1.0,   -25.75,   0.0,
        -1.0,    -1.0,    -1.0,    -1.0,    -1.0]
      )

  Q_expected = np.array(
     [[  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,    3.95 ,   -1.9  ],
      [   3.95 ,   25.0  ,    3.95 ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [   0.0  ,    0.0  ,    0.0  ,    0.0  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [   0.0  ,    0.0  ,    0.0  ,    0.0  ],
      [ -24.175,  -24.175,  -18.325, -100.0  ],
      [  25.0  , -100.0  ,  -18.325,  -24.175],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,  -24.175,   -1.9  ],
      [ -24.175,  -24.175, -100.0  ,   -1.9  ],
      [   0.0  ,    0.0  ,    0.0  ,    0.0  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ],
      [  -1.9  ,   -1.9  ,   -1.9  ,   -1.9  ]]
      )

  Q_actual = calc_Q(mdp, V_test)

  np.testing.assert_almost_equal(Q_expected, Q_actual, decimal=3)

Fail

F
FAIL: test_calcQ (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 40, in test_calcQ
  File "/usr/lib/python3.10/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/numpy/testing/_private/utils.py", line 521, in assert_almost_equal
    return assert_array_almost_equal(actual, desired, decimal, err_msg)
  File "/usr/lib/python3.10/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/numpy/testing/_private/utils.py", line 1034, in assert_array_almost_equal
    assert_array_compare(compare, x, y, err_msg=err_msg, verbose=verbose,
  File "/usr/lib/python3.10/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/numpy/testing/_private/utils.py", line 797, in assert_array_compare
    raise AssertionError(msg)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

#### Generating a Greedy Policy from $Q$

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Generate a greedy policy based on Q!**

In [97]:
def generate_greedy_policy(Q):
  """ Returns a new policy based on greedy selection over Q.

  Args:
    Q: array estimate of the action-value function
  Returns:
    a new greedy policy function
  """
  def greedy_policy(s, a):
    #################################
    # Add your implementation here! #
    #################################
    pass

  return greedy_policy

In [98]:
greedy_policy = generate_greedy_policy(Q)

Let's display our greedy policy. Does it make sense given the $Q$ values we calculated earlier?

In [99]:
display_policy(greedy_policy, action_map=lambda x: ['N', 'S', 'E', 'W'][x])

[['N' 'N' 'N' 'N' 'N']
 ['N' 'N' 'N' 'N' '-']
 ['N' 'N' '-' 'N' 'N']
 ['N' 'N' 'N' 'N' '-']
 ['N' 'N' 'N' 'N' 'N']]


In [100]:
display_state_values(V, decimals=3)

[[ -9.244  -7.538  -4.737   0.402  10.164]
 [-10.614 -12.552 -11.457 -11.008   0.   ]
 [-10.319 -16.721   0.    -64.009 -43.422]
 [-10.166 -23.876 -37.198 -62.608   0.   ]
 [-10.086 -10.045 -10.024 -10.014 -10.009]]


In [101]:
display_action_values(Q, decimals=10)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [102]:
%%unittest_testcase
def test_generate_greedy_policy(self):

  Q_test = np.array(
    [[  -9.0,  -11.0,   -8.0,   -9.0],
      [  -8.0,   -8.0,   -5.0,   -9.0],
      [  -5.0,   -5.0,   -1.0,   -8.0],
      [  -1.0,   -1.0,    8.0,   -5.0],
      [   8.0,   25.0,    8.0,   -1.0],
      [  -9.0,  -10.0,  -12.0,  -11.0],
      [ -12.0,  -16.0,  -11.0,  -11.0],
      [ -11.0,  -11.0,  -11.0,  -12.0]]
  )

  greedy_policy = generate_greedy_policy(Q)

  expected = np.eye(len(mdp.A))[[2, 2, 2, 2, 1, 0, 3, 2]]
  actual = np.array([[greedy_policy(s, a) for a in mdp.A] for s in range(Q_test.shape[0])])

  np.testing.assert_equal(expected, actual)

Fail

F
FAIL: test_generate_greedy_policy (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 19, in test_generate_greedy_policy
  File "/usr/local/lib/python3.10/dist-packages/numpy/testing/_private/utils.py", line 282, in assert_equal
    return assert_array_equal(actual, desired, err_msg, verbose)
  File "/usr/local/lib/python3.10/dist-packages/numpy/testing/_private/utils.py", line 920, in assert_array_equal
    assert_array_compare(operator.__eq__, x, y, err_msg=err_msg,
  File "/usr/lib/python3.10/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/numpy/testing/_private/utils.py", line 797, in assert_array_compare
    raise AssertionError(msg)
AssertionError: 
Arrays are not equal

Mismatched elements: 32 / 32 (100%)
 x: array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],...
 y: array([[None, None, No

<unittest.runner.TextTestResult run=1 errors=0 failures=1>

#### Identifying "Stable" Policies?

It can be shown (see Sutton & Barto, 2020, pp. 78-79) that the policies resulting from each iteration of the Policy Iteration algorithm must be strictly better than the policies from the previous iterations unless the policy is optimal.

We can use this idea to define our convergence criterion for the Policy Iteration algorithm: if the greedy policies generated between two subsequent iterations of the Policy Iteration algorithm would result in the **same selection of actions for every state**, then those policies must be optimal; therefore, we should terminate the algorithm.

When policies fail to improve between iterations they are called **stable** policies.

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Implement a function that determines whether two policies are stable.**

In [103]:
def is_stable(mdp, p1, p2):
  """ Returns whether these two policies are stable.

  Args:
    mdp: the MDP
    p1: 1st policy to compare
    p2: 2nd policy to compare

  Returns:
    True if stable; False otherwise
  """
  S, A, *_ = mdp

  #################################
  # Add your implementation here! #
  #################################

  return True

In [104]:
%%unittest_testcase
def test_stable(self):
  maxes = np.random.default_rng().integers(0, len(mdp.A), size=len(mdp.S))
  maxes_as_prob = np.eye(len(mdp.A))[maxes]

  def policy1(s, a):
    return maxes_as_prob[s][a]

  def policy2(s, a):
    return maxes_as_prob[s][a]

  self.assertTrue(is_stable(mdp, policy1, policy2))

def test_not_stable(self):
  p1_maxes = np.random.default_rng().integers(0, len(mdp.A), size=len(mdp.S))

  # copy array and perturb one element so policies are not identical
  p2_maxes = np.empty_like(p1_maxes)
  p2_maxes[:] = p1_maxes
  p2_maxes[5] = (p2_maxes[5] - 1) % len(mdp.A)

  def policy1(s, a):
    return np.eye(len(mdp.A))[p1_maxes][s][a]

  def policy2(s, a):
    return np.eye(len(mdp.A))[p2_maxes][s][a]

  self.assertFalse(is_stable(mdp, policy1, policy2))

Fail

F.
FAIL: test_not_stable (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 27, in test_not_stable
AssertionError: True is not false

----------------------------------------------------------------------
Ran 2 tests in 0.002s

FAILED (failures=1)


<unittest.runner.TextTestResult run=2 errors=0 failures=1>

Putting it all together!

#### Solution

In [105]:
def policy_improvement(mdp, V, plcy_prev):
  """ Returns an updated policy based on the last state-value estimates.

  Once the improvement step generates a "stable" policy, the algorithm has
  converged.

  Args:
    mdp: the MDP
    V: initial approximation for the state-values
    plcy_prev: the max abs difference termination threshold

  Returns:
    A tuple where the first element is the updated policy and the second element
    is a Boolean indicating whether the new policy is stable.
  """
  Q = calc_Q(mdp, V)
  plcy_new = generate_greedy_policy(Q)

  return plcy_new, is_stable(mdp, plcy_prev, plcy_new)

*Note: The implementation in Sutton & Barto (p. 81) assumes a deterministic policy, which simplifies the notation to $\pi(s)$ instead of $\pi(a|s)$. This assumption is valid because we are defining a greedy policy and choosing the first action to take in the event of a tie. That said, the algorithm works well for stochastic policies $\pi(a|s)$ as well, and the only difference between what we are implementing here and the algorithm described in Sutton & Barto is that we need to loop over all actions, whereas Sutton & Barton's algorithm only needs to evaluate a single action per state. In short, so long as all suboptimal actions are given probability 0.0, then the algorithm is guaranteed to converge. (See Sutton & Barto, p. 79.)*

In [106]:
def policy_iteration(mdp, theta=1e-9, eval_limit=np.inf,
                     verbose=True, debug=False):
  """ Returns the optimal policy and state-value function for this MDP.

  This is an iterative algorithm that terminates when a stable policy is found.

  Args:
    mdp: the MDP
    theta: the max abs difference termination threshold used for policy eval
    eval_limit: the max number of policy eval iterations
    verbose: if True displays summary info per iteration
    debug: if True displays the current V estimates per iteration
  Returns:
    A tuple where the first element is the number of iterations,
    the second element is an array containing optimal state-values
    for every state in mdp, and the third element is the optimal policy.
  """
  k = 0
  V = np.zeros(len(mdp.S))

  # initial policy is uniform random
  policy = generate_uniform_random_policy(mdp)

  stable = False
  while not stable:
    if verbose:
      print(f'beginning iteration {k}', flush=True)

    _, V = policy_evaluation(mdp, policy, update_state_value_estimate, V,
                             theta, eval_limit, verbose=False, debug=False)

    if debug:
      display_state_values(V)

    policy, stable = policy_improvement(mdp, V, policy)

    k += 1

  return k, V, policy

In [107]:
k, V, policy = policy_iteration(mdp, debug=False)

beginning iteration 0


In [108]:
display_state_values(V, 3)

[[ -9.244  -7.538  -4.737   0.402  10.164]
 [-10.614 -12.552 -11.457 -11.009   0.   ]
 [-10.319 -16.721   0.    -64.009 -43.422]
 [-10.166 -23.876 -37.198 -62.608   0.   ]
 [-10.086 -10.045 -10.024 -10.014 -10.01 ]]


In [109]:
display_action_values(calc_Q(mdp, V))

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [110]:
display_policy(policy, action_map)

[['N' 'N' 'N' 'N' 'N']
 ['N' 'N' 'N' 'N' '-']
 ['N' 'N' '-' 'N' 'N']
 ['N' 'N' 'N' 'N' '-']
 ['N' 'N' 'N' 'N' 'N']]


In [111]:
%%unittest_testcase
def test_policy_iteration(self):
  S, A, *_ = mdp
  V_expected = np.array(
    [12.964,  15.515,  18.35,   21.5,   25.0,
    10.667,     8.6,   6.74,  5.066,    0.0,
        8.6,  10.667,    0.0,   21.5,   25.0,
      6.74,  12.964, 15.515,  18.35,    0.0,
      5.066,    3.56,  2.204,  0.983, -0.115])

  def policy_expected(s, a):
    actions = np.array(
        [2, 2, 2, 2, 1,
         0, 1, 3, 3, 0,
         0, 1, 0, 2, 0,
         0, 2, 2, 0, 0,
         0, 3, 3, 3, 3])

    return np.eye(len(A))[actions][s][a]

  k, V_actual, policy_actual = policy_iteration(mdp, verbose=False)

  for s in S:
    for a in A:
      self.assertEqual(policy_actual(s, a), policy_expected(s,a))

Fail

F
FAIL: test_policy_iteration (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 24, in test_policy_iteration
AssertionError: None != 0.0

----------------------------------------------------------------------
Ran 1 test in 5.058s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

#### Verifying Our Solution

*Hint: Read Sutton & Barto, Sec. 3.6 carefully.*

In [112]:
def verify_optimal_state_values(mdp, V, tolerance=1e-5):
  """ Verify the V for an optimal policy satisfies the Bellman optimality equation.

  Args:
    V: array estimates for the optimal state-value function
    tolerance: absolute tolerance on equality check

  Return:
    True if all values in V satisfy the Bellman optimality equation; False
    otherwise.

  """
  #################################
  # Add your implementation here! #
  #################################

  pass

In [113]:
# do not change this line!
%%unittest
assert verify_optimal_state_values(mdp, V)

Fail

F
FAIL: test_1 (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 1, in test_1
AssertionError: None is not true

----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

## Value Iteration

Waiting for the Policy Evaluation step of the Policy Iteration algorithm to converge before each Policy Improvement step can be extremely time consuming. A key insight (see Sutton & Barto, 2020, Sec. 4.4) is that its not necessary to wait for Policy Evaluation to converge before applying the Policy Improvement step.

In the extreme, we can execute a single iteration of Policy Evaluation for each Policy Improvement. This idea is the basis for the **Value Iteration** algorithm.

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Complete the implementation for the Value Iteration algorithm given below.**

In [114]:
def value_iteration(mdp, theta=1e-9, verbose=True, debug=False):
  """ Returns the optimal policy and state-value function for this MDP.

  Args:
    mdp: the MDP
    theta: the max abs difference termination threshold used for convergence
    verbose: if True displays summary info per iteration
    debug: if True displays the current V estimates per iteration
  Returns:
    A tuple where the first element is the number of iterations,
    the second element is an array containing optimal state-values
    for every state in mdp, and the third element is the optimal policy.
  """
  S, A, p, r, gamma = mdp
  k = 0
  delta = np.inf

  V_prev = np.zeros(len(S))

  while delta > theta:
    if verbose:
      print(f'beginning iteration {k}; delta: {delta}', flush=True)

    V = np.zeros(len(S))

    #################################
    # Add your implementation here! #
    #################################

    if debug:
      display_state_values(V, 2)

    delta = np.amax(np.abs(V - V_prev))
    V_prev[:] = V

    k += 1

  return k, V, generate_greedy_policy(calc_Q(mdp, V))

In [115]:
k, V, policy = value_iteration(mdp)

beginning iteration 0; delta: inf


In [116]:
Q = calc_Q(mdp, V)
display_action_values(Q)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [117]:
display_state_values(V, 3)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [118]:
display_policy(policy, action_map)

[['N' 'N' 'N' 'N' 'N']
 ['N' 'N' 'N' 'N' '-']
 ['N' 'N' '-' 'N' 'N']
 ['N' 'N' 'N' 'N' '-']
 ['N' 'N' 'N' 'N' 'N']]


In [119]:
%%unittest_testcase
def test_value_iteration(self):
  S, A, *_ = mdp
  V_expected = np.array(
    [12.964,  15.515,  18.35,   21.5,   25.0,
    10.667,     8.6,   6.74,  5.066,    0.0,
        8.6,  10.667,    0.0,   21.5,   25.0,
      6.74,  12.964, 15.515,  18.35,    0.0,
      5.066,    3.56,  2.204,  0.983, -0.115])

  def policy_expected(s, a):
    actions = np.array(
        [2, 2, 2, 2, 1,
         0, 1, 3, 3, 0,
         0, 1, 0, 2, 0,
         0, 2, 2, 0, 0,
         0, 3, 3, 3, 3])

    return np.eye(len(A))[actions][s][a]

  k, V_actual, policy_actual = value_iteration(mdp, verbose=False)

  for s in S:
    for a in A:
      self.assertEqual(policy_actual(s, a), policy_expected(s,a))

Fail

F
FAIL: test_value_iteration (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 24, in test_value_iteration
AssertionError: None != 0.0

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

Finally, let's compare Value Iteration to Policy Iteration when only a single evaluation step is taken (that is, when `eval_limit=1`.

In [120]:
k, V, policy = policy_iteration(mdp, eval_limit=1)

beginning iteration 0


In [121]:
display_policy(policy, action_map)

[['N' 'N' 'N' 'N' 'N']
 ['N' 'N' 'N' 'N' '-']
 ['N' 'N' '-' 'N' 'N']
 ['N' 'N' 'N' 'N' '-']
 ['N' 'N' 'N' 'N' 'N']]


In [122]:
display_state_values(V)

[[ -1.    -1.    -1.    -1.     5.5 ]
 [ -1.    -1.    -1.    -1.     0.  ]
 [ -1.    -1.     0.   -25.75 -19.25]
 [ -1.    -1.    -1.   -25.75   0.  ]
 [ -1.    -1.    -1.    -1.    -1.  ]]


In [123]:
k, V, policy = value_iteration(mdp)

beginning iteration 0; delta: inf


In [124]:
display_policy(policy, action_map)

[['N' 'N' 'N' 'N' 'N']
 ['N' 'N' 'N' 'N' '-']
 ['N' 'N' '-' 'N' 'N']
 ['N' 'N' 'N' 'N' '-']
 ['N' 'N' 'N' 'N' 'N']]


In [125]:
display_state_values(V)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


#### Verifying Our Solution

In [126]:
# do not change this line!
%%unittest
assert verify_optimal_state_values(mdp, V)

Fail

F
FAIL: test_1 (__main__.JupyterTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 1, in test_1
AssertionError: None is not true

----------------------------------------------------------------------
Ran 1 test in 0.008s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

## Generalized Policy Iteration

Value iteration is the logically the same as Policy Iteration with only single a evaluation step per policy improvement.

However, in general, we can choose different limits to the number of evaluation iterations and get different convergence behavior.

Play around with the `policy_iteration()` function you implemented above and see how the number of evaluation steps (set using the `eval_limit` parameter) affects the convergence of the algorithm.

### Plot Policy Improvement Iterations vs. Policy Evaluation Iterations

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">



**Run `policy_iteration()` multiple times, setting `eval_limit` to each value in the set `{1,2,5,10}`, and then plot 'Policy Evaluation Iterations' vs. 'Policy Improvement Iterations'**

In [127]:
# add code to run policy_iteration where eval_limit = 1,2,5,10

In [128]:
# add code to plot your results

##Markov Decision Process #2 - And Now With Teleporters!

Now it's your turn to formalize a sequential decision problem as a Markov Decision Process (MDP)!

The problem shown below is an updated $5\times5$ grid world that has new wall boundaries, new pit locations, and teleporters!

Using a teleporter requires that agent's execute a "USE(TELEPORTER)" action from a cell containing a teleporter. If this action is taken, then the agent will be wisked away to the teleporter's pair and reappear in that state (see diagram).

<img src="https://drive.google.com/uc?export=view&id=1qx3P7Ipqs8r7nxKLh7s4hbE942ljILGP" width=1200 height=600/>




**Teleporter Dynamics and Rewards**

90% of the time, the teleport will transport the agent to the other teleporter, where you incur the usual $r =-1$ reward for taking an action; *however*, there is a 10% chance that the teleporter will **malfunction**, not only failing to teleport the agent to the destination teleporter (that is, it remains in the same state), but it also causes a painful burning sensation ($r = -10)$!

Take note! This is the first **non-deterministic** state transition and reward for our grid world.

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Define the Markov Decision Process (MDP) for the Problem Given Below**

This requires you to,

1. define the states,
2. define the actions,
3. define the environment's dynamics by implementing `p()`, and
4. define the environment's reward function by implement `r()`.


### States

In [129]:
# add code to define the states of the MDP

### Actions

In [130]:
# add code to define the actions of the MDP

### Environment Dynamics: $p(s'|s,a)$

In [131]:
# update hit_walls function, which will be used to define the environments
# dynamics
def hit_wall(s, a):
  """ Returns true if taking an action a in state s would hit a wall.

  Args:
    s: the state that action a is taken from
    a: the action taken

  Returns:
    True if agent would hit a wall and False otherwise.
  """

  #################################
  # Add your implementation here! #
  #################################

  # didn't hit a wall
  return False

**Implement the environment dynamics function, $p(s'|s,a)$!**

Hints:

1. Refer to the MDP we defined in the previous example to guide you.
2. Use the `hit_wall()` function you defined above to simplify your implementation of `p()`.
3. The environment is now non-deterministic (when using the teleporter), so special care needs to be taken in this case.


Note: Even though the environment's dynamics are non-deterministic, we **do not** need to add a parameter $r$ to `p()`. Given $s'$, $s$, and $a$, the probability of receiving a particular $r$ value is still always 1.0. That is,
rewards are still deterministic given a particular state transition, even though the state transitions themselves are not.

In [132]:
# implement p(s'|s,a) to define the environment dynamics
def p(s, s_prime, a):
  """ Returns the probability of moving from one state to another for an action.

  Args:
    s_prime: the successor state resulting from the action
    s: the state in which the action was taken
    a: the action taken

  Returns:
    the transition probability, p(s'|s,a) in {0.0, 1.0}
  """

  #################################
  # Add your implementation here! #
  #################################

  pass

### Reward Runction: $r(s, a, s')$

In [133]:
# implement r(s',s,a) to define the environment's reward function
def r(s, s_prime, a):
  """ Returns the reward for a single environmental interaction.

  Args:
    s: the state before the action was taken
    s_prime: the state after the action was taken
    a: the action taken
  Returns:
    an immediate reward based on this environmental interaction
  """

  # Add your implementation here!

  pass

In [134]:
# create an instance of the updated MDP
mdp = MDP(states, actions, p, r, gamma)

NameError: name 'gamma' is not defined

### Solution

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Use the algorithms we implemented in the previous section to solve this MDP!**

*Note: A complete solution should include the optimal state-value function and the optimal policy for the MDP.*

*You should display both of these using the provided helper functions.*

In [ ]:
# Add whatever code (and cells) you need to solve the MDP, verify the solution,
# and display stuff!

## Extra Credit

In general, performing calculations in Python is very *slow*.

We can often dramatically improve the runtime of our algorithms by using matrices and vectors instead of nested loops like we did above. Doing so  allows us to pass a large data set to a library like NumPy, which can then perform a series of vectorized operations on that data (e.g., matrix multiplications) in C.

In [ ]:
# utility function
def display_2d_matrix(matrix, fmt=lambda x:x):
  """ Pretty prints a 2D matrix, optimally applying a formatting function.

  Args:
    matrix: a 2D numpy array
    fmt: a function to applied to each cell's value before printing (optional)

  Returns:
    None
  """
  for row in matrix:
    print('\t'.join([str(fmt(col)) for col in row]))

### Updated MDP using Matrices

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Transform each element of the MDP into a matrix or vector (as appropriate)**

In [ ]:
# define matrix P
P = ...

In [ ]:
# define matrix R
R = ...

In [ ]:
# initialize your mdp!
mdp = MDP(S, A, P, R, gamma)

### Policy Evaluation with Vectorized Operations

Hints:

1. Create a new `update_state_value_estimate_vec_ops()` function and pass that into `policy_evalution()`'s `update` parameter.
2. Perform matrix multiplications, dot products, etc. (as needed) inside of `update_state_value_estimate_vec_ops()` to perform the Bellman expectation equation's update.
3. Test your solution to make sure you get the same answers as you did with the non-vectorized implementation!

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Now rewrite the update rule for Policy Evaluation to use vectorized operations.**

In [ ]:
def update_state_value_estimate_vec_ops(mdp, V_prev, policy):
  """ Returns an updated estimate of the state-values.

  Args:
    mdp: the MDP (in vectorized form)
    V_prev: an array of state-value estimates from the previous iteration
    policy: the policy being evaluated

  Returns:
    An array containing updated state-values for every state in the MDP.
  """

  # P and R are now matrices!
  S, A, P, R, gamma = mdp

  V = np.zeros(len(states))


  #################################
  # Add your implementation here! #
  #################################


  return V

In [ ]:
# notice that the update function that we are passing in is the function with
# vectorized operations
_ = policy_evaluation(mdp, uniform_random_policy,
                      update_state_value_estimate_vec_ops,
                      k_max=5, debug=True)

### Value Iteration with Vectorized Operations

Finally, rewrite the Value Iteration algorithm to use vectorized operations.

This will require changing two functions
1. `calc_Q()`
2. `value_iteration()`

Stubs for the vectorized versions are provided below.

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Now rewrite the `calc_Q()` to use vectorized operations.**

In [ ]:
def calc_Q_with_vec_ops(mdp, V):
  """ Calculates an array of action-values from state-value estimates.

  Args:
    mdp: the MDP
    V: an array of state-value estimates

  Return:
    An array of action-value estimates.
  """
  # P and R are now matrices!
  S, A, P, R, gamma = mdp

  Q = np.zeros(shape=(len(states), len(actions)))

  #################################
  # Add your implementation here! #
  #################################

  return Q

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Now rewrite the `value_iteration()` to use vectorized operations.**

In [ ]:
def value_iteration_with_vec_ops(mdp, theta=1e-9, verbose=True, debug=False):
  """ Returns the optimal policy and state-value function for this MDP.

  Args:
    mdp: the MDP
    theta: the max abs difference termination threshold used for convergence
    verbose: if True displays summary info per iteration
    debug: if True displays the current V estimates per iteration
  Returns:
    A tuple where the first element is the number of iterations,
    the second element is an array containing optimal state-values
    for every state in mdp, and the third element is the optimal policy.
  """
  # P and R are now matrices!
  S, A, P, R, gamma = mdp
  k = 0
  delta = np.inf

  V_prev = np.zeros(len(S))

  while delta > theta:
    if verbose:
      print(f'beginning iteration {k}; delta: {delta}', flush=True)

    V = np.zeros(len(S))

    #################################
    # Add your implementation here! #
    #################################

    if debug:
      display_state_values(V, 2)

    delta = np.amax(np.abs(V - V_prev))
    V_prev[:] = V

    k += 1

  return k, V, generate_greedy_policy(calc_Q_with_vec_ops(mdp, V))

### Verify Solutions

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Write test cases to verify your new implementations work correctly!**

In [ ]:
# Your functional test cases here!

### Timing Results

Now use `%%timeit` to compare the performance of your vectorized implementation of `policy_evaluation()` to the earlier, non-vectorized implementation.

That is, compare the timing of `policy_evaluation()` using `update=update_state_value_estimate` and `update=update_state_value_estimate_vec_ops`.

<img src="https://img.freepik.com/premium-vector/scientist-professor-holding-stop-sign_20412-3130.jpg" width="225" height="200">

**Write performance tests!**

In [ ]:
# Your performance tests and results here!